# RPLib Problem 0001
## Baseline

Goal of this notebook is to analyze and visualize the results

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [3]:
from pathlib import Path
home = str(Path.home())
home

'/home/jupyter-pander14'

In [4]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [5]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
from sensitivity_tests import *
from utilities import *
from base import *

In [6]:
import itertools
import joblib

In [7]:
problem = joblib.load("/disk/RPLib/problem_0001.joblib.z")

In [8]:
print(problem["description"])


A practitioner wants to predict the degree to which a the rankings during season 
of the NCAA Men’s Basketball are likely to change as more games are played (i.e., sensitivity to more games). 
They want to start the analysis after a minimum of 50% of the games are played. 
They want to run Massey and Colley.

Sensitivity of new games will be measured as the intersection of between two 
rankings derived from before and after the new games are included.



In [9]:
problem['target']

,frac1,frac2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,top10_intersection
0,0.5,0.6,all,madness,1.0,3.0,0.25,Massey,2002,0.2
1,0.5,0.6,all,madness,1.0,3.0,0.25,Colley,2002,0.4
2,0.5,0.7,all,madness,1.0,3.0,0.25,Massey,2002,0.1
3,0.5,0.7,all,madness,1.0,3.0,0.25,Colley,2002,0.3
4,0.5,0.8,all,madness,1.0,3.0,0.25,Massey,2002,0.1
...,...,...,...,...,...,...,...,...,...,...
1015,0.8,0.9,madness,madness,1.0,3.0,0.25,Colley,2018,0.1
1016,0.8,1.0,madness,madness,1.0,3.0,0.25,Massey,2018,0.4
1017,0.8,1.0,madness,madness,1.0,3.0,0.25,Colley,2018,0.2
1018,0.9,1.0,madness,madness,1.0,3.0,0.25,Massey,2018,0.2


In [10]:
problem['data'].keys()

dict_keys(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [11]:
problem['data']['2002'].keys()

dict_keys(['frac=0.5', 'frac=0.6', 'frac=0.7', 'frac=0.8', 'frac=0.9', 'frac=1.0'])

In [12]:
years = list(problem['data'].keys())
frac_keys = list(problem['data'][years[0]].keys())
remaining_games = problem['other']['remaining_games']
madness_teams = problem['other']['madness_teams']

In [44]:
# Parameters to search
direct_thress = [0]
spread_thress = [0]
weight_indirects = [0.25]
domains_ranges = [('all','madness')]

In [45]:
def compute_D(game_df,team_range,direct_thres,spread_thres,weight_indirect):
    map_func = lambda linked: pyrankability.construct.support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
    D = pyrankability.construct.V_count_vectorized(game_df,map_func).reindex(index=team_range,columns=team_range)
    return D

In [156]:
feature_columns = ["delta_lop","delta_hillside","nfrac_xstar_lop","nfrac_xstar_hillside","diameter_lop","diameter_hillside"]

def calc_tau(n,obj):
    nchoose2 = pyrankability.common.nCr(n,2)
    tau = (nchoose2 - obj)/nchoose2
    return tau

def compute_features(D):
    delta_lop,details_lop = pyrankability.rank.solve(D.fillna(0),method='lop',cont=True)

    x = pd.DataFrame(details_lop['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_lop = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D.fillna(0),method='lop',verbose=False)
    d_lop = calc_tau(len(D),details_two_distant['obj'])
    
    delta_hillside,details_hillside = pyrankability.rank.solve(D,method='hillside',cont=True)
    
    x = pd.DataFrame(details_hillside['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_hillside = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D,method='hillside',verbose=False)
    d_hillside = calc_tau(len(D),details_two_distant['obj'])
    
    features = pd.Series([delta_lop,delta_hillside,2*nfrac_upper_lop,2*nfrac_upper_hillside,d_lop,d_hillside],index=feature_columns)

    return features

In [157]:
target_column = "top10_intersection"
def process(data,target):
    index_cols = ["Year","frac_key","direct_thres","spread_thres","weight_indirect","range"]
    Ds = pd.DataFrame(columns=["D"]+index_cols)
    Ds.set_index(index_cols,inplace=True)
    outer_keys = list(itertools.product(years,frac_keys,direct_thress,spread_thress,weight_indirects,domains_ranges))
    for year,frac_key,dt,st,iw,domain_range in tqdm(outer_keys):
       # set the team_range
        team_range = None
        ran = domain_range[1]
        if ran == 'madness':
            team_range = madness_teams[year]
        elif ran == 'all':
            team_range = all_teams[year]
        elif "top" in ran:
            team_range = all_teams[year]
        D = compute_D(data[year][frac_key],team_range,dt,st,iw)
        Ds = Ds.append(pd.Series([D],index=["D"],name=(year,frac_key,dt,st,iw,ran))) 
    return Ds

In [158]:
Ds = process(problem['data'],problem['target'])






  0%|          | 0/102 [00:00<?, ?it/s]




  3%|▎         | 3/102 [00:00<00:04, 22.43it/s]




  6%|▌         | 6/102 [00:00<00:04, 21.82it/s]




  9%|▉         | 9/102 [00:00<00:04, 22.30it/s]




 12%|█▏        | 12/102 [00:00<00:04, 21.78it/s]




 15%|█▍        | 15/102 [00:00<00:03, 22.29it/s]




 18%|█▊        | 18/102 [00:00<00:03, 21.80it/s]




 21%|██        | 21/102 [00:00<00:03, 22.26it/s]




 24%|██▎       | 24/102 [00:01<00:03, 21.77it/s]




 26%|██▋       | 27/102 [00:01<00:03, 22.19it/s]




 29%|██▉       | 30/102 [00:01<00:03, 21.62it/s]




 32%|███▏      | 33/102 [00:01<00:03, 22.05it/s]




 35%|███▌      | 36/102 [00:01<00:03, 21.44it/s]




 38%|███▊      | 39/102 [00:01<00:02, 21.87it/s]




 41%|████      | 42/102 [00:01<00:02, 21.23it/s]




 44%|████▍     | 45/102 [00:02<00:02, 21.71it/s]




 47%|████▋     | 48/102 [00:02<00:02, 21.22it/s]




 50%|█████     | 51/102 [00:02<00:02, 21.74it/s]




 53%|█████▎    | 54/102 [00:02<00:02, 17.88it/s]






In [159]:
Ds.loc[('2002','frac=0.5',0,0,0.25,'madness'),'D']

team2,Alabama,Alcorn_St,Arizona,Boston_College,Boston_Univ,California,Central_Conn,Charlotte,Cincinnati,Connecticut,...,UNC_Wilmington,USC,Utah,Valparaiso,W_Kentucky,Wake_Forest,Winthrop,Wisconsin,Wyoming,Xavier
team1,,,,,,,,,,,,,,,,,,,,,
Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
Alcorn_St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25
Boston_College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boston_Univ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wake_Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Winthrop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
Ds.index.names

FrozenList(['Year', 'frac_key', 'direct_thres', 'spread_thres', 'weight_indirect', 'range'])

In [165]:
def create_features(Ds):
    index_cols = list(Ds.index.names)
    X = pd.DataFrame(columns=index_cols + feature_columns)
    X.set_index(index_cols,inplace=True)
    for index,row in tqdm(Ds.iterrows()):
        year,frac_key,dt,st,iw,ran = index
        features = compute_features(Ds.loc[(year,frac_key,dt,st,iw,ran),"D"])
        features.name = index
        X = X.append(features)
    return X

In [ ]:
X = create_features(Ds)







0it [00:00, ?it/s]





1it [00:34, 34.83s/it]





2it [01:12, 35.58s/it]





3it [01:54, 37.51s/it]





4it [02:35, 38.61s/it]





5it [03:06, 36.26s/it]





6it [03:41, 35.94s/it]

In [ ]:
X

In [ ]:
index_cols = list(Ds.index.names)
index_cols

### Refine the target dataset
We will try to predict the average over the parameters run for each method.

In [ ]:
target = problem['target'].groupby(['frac1','frac2','Method','Year'])['top10_intersection'].mean().to_frame()
target

In [ ]:
X_for_join = X.copy().reset_index()
X_for_join['frac1']= X_for_join['frac_key'].str.replace("frac=","").astype(float)
X_for_join

In [ ]:
Xy = target.reset_index().set_index(['frac1','Year']).join(X_for_join.set_index(['frac1','Year']))
Xy

## Narrowing our goal to 0.5 and 0.6

In [ ]:
data = Xy.reset_index().set_index(['frac1','frac2']).loc[0.5,0.6]
data

In [ ]:
from sklearn.preprocessing import scale
features = ["delta_lop","delta_hillside"]
for_index = list(data.drop(features+['Year']+[target_column],axis=1).columns)
scaled_data = data.copy().reset_index().set_index(for_index)
for ix in scaled_data.index.unique():
    print("Scaling for group of",ix)
    scaled_data.loc[ix,features] = scale(scaled_data.loc[ix,features])
scaled_data = scaled_data.reset_index()
scaled_data[target_column] = data[target_column].values
scaled_data

In [ ]:
import sklearn.linear_model as skl_lm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold

# based on 5x2 from http://ieeexplore.ieee.org/document/6790639/
# Dietterich also recommends a resampling method of his own devising called 5×2 cross-validation that involves 5 repeats of 2-fold cross-validation.
# Two folds are chosen to ensure that each observation appears only in the train or 
# test dataset for a single estimate of model skill. A paired Student’s t-test is used 
# on the results, updated to better reflect the limited degrees of freedom given the 
# dependence between the estimated skill scores.
# This interval is 2.571 for a 5% threshold and 3.365 for a 2% thresholds (https://www.medcalc.org/manual/t-distribution.php)

def evaluate(df,pred_col,model1,model2,param_grid1={},param_grid2={},feature_cols=["delta_lop","delta_hillside"]):
    trn = df[feature_cols]
    target = df[pred_col]
    # Choose seeds for each 2-fold iterations
    seeds = [13, 51, 137, 24659, 347]
    # Initialize the score difference for the 1st fold of the 1st iteration 
    p_1_1 = 0.0
    # Initialize a place holder for the variance estimate
    s_sqr = 0.0
    # Initialize scores list for both classifiers
    scores_1 = []
    scores_2 = []
    diff_scores = []
    # Iterate through 5 2-fold CV
    for i_s, seed in enumerate(seeds):
        # Split the dataset in 2 parts with the current seed
        folds = KFold(n_splits=2, shuffle=True, random_state=seed)
        # Initialize score differences
        p_i = np.zeros(2)
        # Go through the current 2 fold
        for i_f, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
            # Split the data
            trn_x, trn_y = trn.iloc[trn_idx], target.iloc[trn_idx]
            val_x, val_y = trn.iloc[val_idx], target.iloc[val_idx]
            cv = [(slice(None), slice(None))] # don't perform any cross validation
            grid1 = GridSearchCV(model1,param_grid1,verbose=0,n_jobs=-1,cv=cv,refit=True)
            grid2 = GridSearchCV(model2,param_grid2,verbose=0,n_jobs=-1,cv=cv,refit=True)
            # Train classifiers
            grid1.fit(trn_x, trn_y)
            grid2.fit(trn_x, trn_y)
            best_estimator1 = grid1.best_estimator_
            best_estimator2 = grid2.best_estimator_
            errors1 = val_y - best_estimator1.predict(val_x).flat
            errors2 = val_y - best_estimator2.predict(val_x).flat
            score_1 = -np.mean(np.abs(errors1))
            score_2 = -np.mean(np.abs(errors2))

            # keep score history for mean and stdev calculation
            scores_1.append(score_1)
            scores_2.append(score_2)
            diff_scores.append(score_1 - score_2)
            #print("Fold %2d score difference = %.6f" % (i_f + 1, score_1 - score_2))
            # Compute score difference for current fold  
            p_i[i_f] = score_1 - score_2
            # Keep the score difference of the 1st iteration and 1st fold
            if (i_s == 0) & (i_f == 0):
                p_1_1 = p_i[i_f]
        # Compute mean of scores difference for the current 2-fold CV
        p_i_bar = (p_i[0] + p_i[1]) / 2
        # Compute the variance estimate for the current 2-fold CV
        s_i_sqr = (p_i[0] - p_i_bar) ** 2 + (p_i[1] - p_i_bar) ** 2 
        # Add up to the overall variance
        s_sqr += s_i_sqr

    # Compute t value as the first difference divided by the square root of variance estimate
    t_bar = p_1_1 / ((s_sqr / 5) ** .5) 
 
    return pd.Series([t_bar,np.mean(diff_scores), np.std(diff_scores),np.mean(scores_1),np.mean(scores_2),np.std(scores_1),np.std(scores_2)],index=["t_bar","Difference Mean","Difference Stdev","Mean Score 1","Mean Score 2","Stdev 1","Stdev 2"])



In [125]:
for_index

['Method',
 'frac_key',
 'direct_thres',
 'spread_thres',
 'weight_indirect',
 'range']

In [130]:
dummy_svr_results = scaled_data.groupby(for_index).apply(evaluate,target_column,DummyRegressor(),SVR(gamma='scale'),param_grid1 = {},
                                                                           param_grid2 = {'C': [0.1,1,10], 
                                                                                   'epsilon': [0.1,0.5,1.],
                                                                                   'kernel': ['linear']#, 'poly', 'rbf', 'sigmoid'],
                                                                                  })


In [131]:
dummy_svr_results

,,,,,,t_bar,Difference Mean,Difference Stdev,Mean Score 1,Mean Score 2,Stdev 1,Stdev 2
Method,frac_key,direct_thres,spread_thres,weight_indirect,range,,,,,,,
Colley,frac=0.5,0,0,0.25,madness,0.001696,0.006281,0.030096,-0.069306,-0.075587,0.015835,0.029062
Massey,frac=0.5,0,0,0.25,madness,0.318519,0.010834,0.015323,-0.072222,-0.083056,0.011275,0.021677


In [132]:
from sklearn.linear_model import LinearRegression

dummy_linear_results = scaled_data.groupby(for_index).apply(evaluate,target_column,DummyRegressor(),
                                                            LinearRegression(),param_grid1 = {},
                                                            param_grid2 = {})


In [133]:
dummy_linear_results

,,,,,,t_bar,Difference Mean,Difference Stdev,Mean Score 1,Mean Score 2,Stdev 1,Stdev 2
Method,frac_key,direct_thres,spread_thres,weight_indirect,range,,,,,,,
Colley,frac=0.5,0,0,0.25,madness,-0.044668,0.028209,0.067776,-0.069306,-0.097515,0.015835,0.064442
Massey,frac=0.5,0,0,0.25,madness,0.291950,0.059057,0.069604,-0.072222,-0.131279,0.011275,0.078396


In [136]:
scaled_data.set_index('Method').loc['Colley'][["top10_intersection","delta_lop","delta_hillside"]].corr()

,top10_intersection,delta_lop,delta_hillside
top10_intersection,1.000000,0.282327,0.332860
delta_lop,0.282327,1.000000,0.971624
delta_hillside,0.332860,0.971624,1.000000
